In [45]:
from bs4 import BeautifulSoup
import requests
import psycopg2
from collections import deque

TIKI_URL = 'https://tiki.vn/'

In [89]:
def parse(url):
    try:
        response = requests.get(url).text
        response = BeautifulSoup(response, "html.parser")
        return response
    except Exception as err:
        print(f'ERROR: {err}')
        return ''

### CATEGORIES SCRAPING

In [87]:
conn = psycopg2.connect(user="nhanpham", database="tikifinal")
conn.autocommit = True
cur = conn.cursor()

def create_category_table():
    query = """
            CREATE TABLE IF NOT EXISTS categories(
                id SERIAL PRIMARY KEY,
                name VARCHAR(255),
                url TEXT,
                parent_id INT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                );
            """
    try:
        cur.execute(query)
    except Exception as err:
        print(f'ERROR: {err}')

In [88]:
class Category:
    def __init__(self, cat_id, name, url, parent_id):
        self.cat_id = cat_id
        self.name = name
        self.url = url
        self.parent_id = parent_id
        
    def save_into_db(self):
        query = f'SELECT url FROM categories WHERE url LIKE %s;'
        val = self.url
        try:
            cur.execute(query, val)
            result = cur.fetchall()
            if len(result) > 0:
                return ''
        except Exception as err:
            print(f'ERROR: {err}')
            
        query = f"""
                INSERT INTO categories (name, url, parent_id) 
                VALUES (%s, %s, %s) RETURNING id;
                """
        val = (self.name, self.url, self.parent_id)
        try:
            cur.execute(query, val)
            #GET ID FROM NEW ROW 
            self.cat_id = cur.fetchone()[0]
        except Exception as err:
            print(f'ERROR: {err}')
        
    def __repr__(self):
        return f'ID: {self.cat_id}, Name: {self.name}, URL: {self.url}, Parent ID: {self.parent_id}'

In [90]:
def get_main_categories(save_db=False):
    s = parse(TIKI_URL)
    category_list = []
    for i in s.findAll('a',{'class':'MenuItem__MenuLink-tii3xq-1 efuIbv'}):
        cat_id = None
        name = i.find('span', {'class':'text'}).text 
        url = i['href'] + "&page=1"
        parent_id = None
        cat = Category(None, name, url, parent_id)
        if save_db:
            cat.save_into_db()
        category_list.append(cat)
        
    return category_list

In [92]:
def get_sub_categories(category, save_db=False):
    name = category.name
    url = category.url
    sub_categories = []

    try:
        div_containers = parse(url).find_all('div', attrs={"class": "list-group-item is-child"})
        for div in div_containers:
            sub_id = None
            sub_name = ' '.join(div.a.text.split()[:-1])
            sub_url = 'https://tiki.vn' + div.a.get('href')
            sub_parent_id = category.cat_id
            
            cat = Category(sub_id, sub_name, sub_url, sub_parent_id)
            if save_db:
                cat.save_into_db()
            if cat.cat_id is not None:
                sub_categories.append(cat)
    except Exception as err:
        print(f'ERROR: {err}')
    
    return sub_categories

In [93]:
def get_all_categories(main_categories):
    queue = deque(main_categories)
    count = 0
    
    while queue:
        parent_cat = queue.popleft()
        sub_list = get_sub_categories(parent_cat, save_db=True)
        queue.extend(sub_list)
        
        # sub_list is empty, which mean the parent_cat has no sub-categories
        if not sub_list:
            count+=1
            if count % 100 == 0:
                print(f'{count} number of deepest nodes')

In [94]:
%%time
create_category_table()
main_categories = get_main_categories(save_db=True)
get_all_categories(main_categories)

100 number of deepest nodes
200 number of deepest nodes
300 number of deepest nodes
400 number of deepest nodes
500 number of deepest nodes
600 number of deepest nodes
700 number of deepest nodes
800 number of deepest nodes
900 number of deepest nodes
1000 number of deepest nodes
1100 number of deepest nodes
1200 number of deepest nodes
1300 number of deepest nodes
1400 number of deepest nodes
1500 number of deepest nodes
1600 number of deepest nodes
1700 number of deepest nodes
1800 number of deepest nodes
1900 number of deepest nodes
2000 number of deepest nodes
2100 number of deepest nodes
2200 number of deepest nodes
2300 number of deepest nodes
2400 number of deepest nodes
2500 number of deepest nodes
2600 number of deepest nodes
CPU times: user 16min 36s, sys: 9.69 s, total: 16min 45s
Wall time: 1h 6min 59s


### PRODUCT SCRAPING

In [100]:
def create_product_table():
    query = """
            CREATE TABLE IF NOT EXISTS products(id SERIAL PRIMARY KEY,
                                                data_id INT,
                                                seller_id INT,
                                                name VARCHAR(255),
                                                price INT,
                                                img TEXT,
                                                cat_id INT,
                                                submaster_cat VARCHAR(255),
                                                submaster_link TEXT,
                                                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                                                );
            """
    try:
        cur.execute(query)
    except Exception as err:
        print(f'ERROR: {err}')

In [101]:
class Product:
    def __init__(self, product_id, data_id, seller_id, name, price, img, cat_id, submaster_cat, submaster_link):
        self.product_id = product_id
        self.data_id = data_id 
        self.seller_id = seller_id
        self.name = name
        self.price = price
        self.img = img 
        self.cat_id = cat_id
        self.submaster_cat = submaster_cat
        self.submaster_link = submaster_link
        
    def save_into_db(self):
        query = f"""
                INSERT INTO products (data_id, seller_id, name, price, img, cat_id, submaster_cat, submaster_link) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s) RETURNING id;
                """
        val = (self.data_id, self.seller_id, self.name, self.price, self.img, self.cat_id, self.submaster_cat, self.submaster_link)
        try:
            cur.execute(query, val)
        except Exception as err:
            print(f'ERROR: {err}')
        
    def __repr__(self):
        return f'ID: {self.product_id}, Data ID: {self.data_id}, Seller ID: {self.seller_id}, Name: {self.name}, Price: {self.price}, IMG: {self.img}, Category ID: {self.cat_id}, Sub Master Category: {self.submaster_cat}, Sub Master Link: {self.submaster_link}'

In [102]:
def scrape_one(cat, sub_url):
    """ return scraped products in one page of certain category
    """
    
    results = []
    s = parse(sub_url)
    product_items = s.findAll('div',{'class':'product-item'})
    if len(product_items) == 0:
        return []
    else: 
        for i in range(len(product_items)):
            row = [product_items[i]['data-id'], 
                   product_items[i]['data-seller-product-id'] if len(product_items[i]['data-seller-product-id']) != 0 else None, 
                   product_items[i]['data-title'],
                   product_items[i]['data-price'], 
                   product_items[i].find('img',{'class':'product-image img-responsive'})['src'], 
                   cat,
                   s.find('div',{'class':'breadcrumb-wrap'}).find('ul').findAll('li')[2].text,
                   'https://tiki.vn' + s.find('div',{'class':'breadcrumb-wrap'}).find('ul').findAll('li')[2].a.get('href') +'?src=tree'
                   ]
            results.append(row)
    return results

In [ ]:
def get_nochild_cate():
    """ return all the categories that have no sub categories
    """
    query = ("""SELECT p.url, p.id 
                FROM categories as p 
                LEFT JOIN categories as c ON c.parent_id = p.id 
                WHERE c.id IS NULL;
                """)
    cur.execute(query)
    sub_cate_list = cur.fetchall()

    for sub_cate in sub_cate_list:
        temp = list(sub_cate)
        temp[0] += '&page=1'
        sub_cate_list[sub_cate_list.index(sub_cate)] = tuple(temp) 
    
    return sub_cate_list 

In [103]:
sub_cate_list = get_nochild_cate()
sub_cate_list[:10]

[('https://tiki.vn/bong-tam-cho-be/c10460?src=tree&page=1', 2848),
 ('https://tiki.vn/usb-luu-tru/c1828?src=tree&page=1', 251),
 ('https://tiki.vn/tinh-chat-duong-am/c11695?src=tree&page=1', 2026),
 ('https://tiki.vn/presentation/c76?src=tree&page=1', 3028),
 ('https://tiki.vn/business-economics/c21306?src=tree&page=1', 2409),
 ('https://tiki.vn/sua-duong-am/c11691?src=tree&page=1', 2024),
 ('https://tiki.vn/modem-adsl/c4585?src=tree&page=1', 264),
 ('https://tiki.vn/luoc-cho-be/c10459?src=tree&page=1', 2847),
 ('https://tiki.vn/o-cung-hdd/c4051?src=tree&page=1', 1350),
 ('https://tiki.vn/thiet-bi-so-khac/c21440?src=tree&page=1', 1070)]

In [ ]:
def scrape_all(page_limit=10, product_limit=100000):
    """ return scrape products from all the last-level categories
        (categories without children)
    """
    
    print('INFO scrape_all(): Start scraping')

    queue = sub_cate_list 
    while queue:
        url = queue[0][0]
        cat = queue[0][1]
        queue = queue[1:]
        
        results = []
        new_rows = scrape_one(cat, url)
        if new_rows:
            for product in new_rows:
                id_ = None 
                data_id = product[0]
                seller_id = product[1]
                name = product[2]
                price = product[3]
                img = product[4]
                cat_id = product[5]
                sub_master_cat = product[6]
                sub_master_link = product[7]

                product = Product(id_, data_id, seller_id, name, price, img, cat_id, sub_master_cat, sub_master_link)
                product.save_into_db()
                results += new_rows 
        
        # Generate next page url 
            page = int(url[-1]) + 1
            url = url[:-1] + str(page)
            
        # Checking limit
            if page < page_limit:
                queue.append((url,cat))
            query = 'SELECT COUNT(*) FROM products'
            cur.execute(query) 
            
            if cur.fetchall()[0][0] > product_limit:
                print('Task completed!')
                return results

In [ ]:
create_product_table()
scrape_all()